# 문자열 데이터 전처리

## #01. Colab에 Mecab-kor-doc 설치하기

한글 형태소 분석을 수행할 경우 새로운 노트북 생성시마다 해 줘야 한다.

참고 : https://github.com/SOMJANG/Mecab-ko-for-Google-Colab

In [ ]:
# !git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
# !bash Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab_light_220429.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 138 (delta 26), reused 22 (delta 8), pack-reused 91
Receiving objects: 100% (138/138), 1.72 MiB | 11.05 MiB/s, done.
Resolving deltas: 100% (65/65), done.
Installing konlpy.....
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 41.0 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-09-14 06:02:55--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuse

## #02. 패키지

In [ ]:
from konlpy.tag import Mecab
from tensorflow.keras.preprocessing.text import Tokenizer

## #03. 문자열 토큰화

학습 데이터에서 `단어 단위`로 `일련번호로 변환`하는 처리

### 1. Token (=형태소)

- 문법적으로 더 이상 나눌 수 없는 언어 요소.
- 영어의 경우 각 단어로 나누면 되지만 한글의 경우 복잡한 처리 과정을 거쳐야 하기 때문에 별도의 라이브러리를 적용해야 한다. (konlpy, mecab 등)

### 2. 영어 문장에 대한 토큰화

#### 토큰화 학습 데이터

In [ ]:
train_text = ["You are the Best", "You are Nice"]

#### 토큰화 객체 생성

- `num_words` : 밀집 표현의 최대 벡터 길이를 지정 (= 최대 단어 수)
- `oov_token` : 학습 결과를 적용할 때 학습 데이터에 포함되지 않은 단어가 존재할 경우 대체할 단어.

In [ ]:
tokenizer = Tokenizer(num_words=10, oov_token="<OOV>")

#### 토큰화 학습

In [ ]:
# 토큰화 학습
tokenizer.fit_on_texts(train_text)

# 각 단어에 부여된 index 번호 확인
print(tokenizer.word_index)

{'<OOV>': 1, 'you': 2, 'are': 3, 'the': 4, 'best': 5, 'nice': 6}


#### 학습 결과 적용

토큰화를 학습한 모델에 `새로운 단어를 적용`하여 `밀집 벡터로 변환`한다.

In [ ]:
train_text = ["We are the Best", "We are Nice"]

In [ ]:
sequences = tokenizer.texts_to_sequences(train_text)
print(sequences)    #We는 학습된 단어가 아니므로 <OOV>:1 할당

[[1, 3, 4, 5], [1, 3, 6]]


### 3. 한글 문장 토큰화

형태소 분석 엔진 Mecab을 설치해야 한다.

#### 토큰화 학습 데이터

In [ ]:
poem = """
흘러내린 머리카락이 흐린 호박빛 아래 빛난다.
난 유영한다. 차분하게 과거에 살면서 현재의 공기를 마신다.
가로등이 깜빡인다.
나도 깜빡여준다.
"""

#### 불용어(stopwords) 목록

`분석에서 제외``할 불필요한 단어에 대한 목록.

개발자가 `임의로 지정, 불용어 사전을 웹에서 내려받아 사용`할 수 있다.

In [ ]:
stopwords = ["난", "나"]

#### 형태소 분석을 통해 명사만 추출

In [ ]:
mecab = Mecab()
nouns = mecab.nouns(poem)   #Mecab을 사용해 poen에서 noun만을 추출
print(nouns)

['머리카락', '호박', '빛', '아래', '난', '유영', '과거', '현재', '공기', '가로등', '나']


#### 추출된 명사에서 `불용어를 제외한 새로운 리스트` 만들기

In [ ]:
train_text = []

for i in nouns:
    if i not in stopwords:
        train_text.append(i)
train_text

['머리카락', '호박', '빛', '아래', '유영', '과거', '현재', '공기', '가로등']

#### 토큰화

In [ ]:
tokenizer = Tokenizer(num_words = len(nouns), oov_token="<OOV>")
tokenizer.fit_on_texts(train_text)
print(tokenizer.word_index)

{'<OOV>': 1, '머리카락': 2, '호박': 3, '빛': 4, '아래': 5, '유영': 6, '과거': 7, '현재': 8, '공기': 9, '가로등': 10}


## #04. 워드 임베딩(Word Embedding)

컴퓨터가 자연어를 이해하고, 효율적으로 처리하게 하기 위해서는 컴퓨터가 이해할 수 있도록 자연어를 적절히 변환할 필요가 있다.

워드 임베딩(Word Embedding)은 단어를 벡터로 표현하는 방법으로, 단어를 희소 표현이 개선된 형태인 밀집 표현으로 변환한다.

### 1. 희소 표현 (원-핫 백터)

원-핫 인코딩과 비슷한 방법

표현하고자 하는 단어의 인덱스 값만 1이고 나머지 인덱스에는 전부 0으로 표현되는 백터 방법으로 원-핫 백터라고도 한다.

#### 예시문장 두 개

```
머신러닝 공부는 재미있다.
머신러닝은 유용하다.
```

#### 형태소 분석

위 의 두 예문에서 명사인 단어만 추출한다면 다음과 같다.

```
머신러닝, 공부, 재미
머신러닝, 유용
```

#### 토큰화

각 단어에 인덱스 번호를 적용한다면 아래와 같이 표현할 수 있을 것이다.

```
0, 1, 2
0, 3
```

#### 희소 표현

전체 단어의 수가 `4`개 이므로 각각의 단어를 4개의 원소를 갖는 리스트 안에서 one-hot 인코딩으로 표현한다.

> one-hot 인코딩으로 표현하면 이미지 형태로 변경 -> softmax 사용 가능


```
[ [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0] ]
[ [1, 0, 0, 0], [0, 0, 0, 1] ]
```

#### 희소 표현의 한계

단어의 개수가 늘어나면 벡터의 차원이 한없이 커진다.

여러 문장을 통해 얻어진 단어가 10,000개이고 그 중에서 5개의 단어로 구성된 문장이라면 벡터의 총 길이는 50,000이 되게 된다.

그러므로 공간적인 낭비를 가져온다.


### 2. 밀집표현

희소 표현의 반대

벡터의 차원을 단어 집합의 크기로 결정하지 않고 분석가가 설정한 임의의 값으로 모든 단어 벡터의 차원을 맞춘다.

`0`과 `1`이 아닌 실수를 원소로 갖는다.

### 앞에서 제시한 예시 문장에서 `머신러닝`이라는 단어의 예시

희소표현 → `[1, 0, 0, 0]`

벡터의 차원을 2로 설정한 밀집표현 → `[1.8, -0.4]`

> 수치 값은 임의의 값이다. 이와 같은 식으로 실수 형태로 표현된다는 것을 표현한 것일 뿐 실제 정확한 값은 아님을 이해하자.

결과적으로 벡터의 차원이 조밀해졌다고 하여 밀집 벡터라고 부른다.

### 3. 워드 임베딩

단어를 밀집 벡터의 형태로 표현한 방법.

케라스에서는 **토큰화** 한 단어 벡터를 `Embedding()` 함수에 전달하여 학습층을 쌓음으로서 적용할 수 있다.

단어를 랜덤한 값을 갖는 밀집 벡터로 변환한 뒤에, 인공 신경망의 가중치를 학습하는 방식으로 단어 벡터를 학습한다.

> 이전까지는 주어진 종속변수 외에 다른 값이 주어지면 정확성이 급격히 하락.

> 즉, 높은 정답율을 위해서는 정답의 가지수를 줄여야함. -> 지도학습(회귀/분류 분석)